### Week 8: Real-World ETL Example
##### Processing E-commerce Order Data

In [4]:
import pandas as pd
from datetime import datetime

# =============================================================================
# SCENARIO: Processing Daily E-commerce Orders
# =============================================================================

"""
You work for an e-commerce company. Every day you receive order data that needs:
1. Validation (remove invalid orders)
2. Categorization (tag orders by size and urgency)
3. Calculations (apply discounts, calculate shipping)
4. Reporting (summarize for management)
"""


# Raw order data (this would normally come from a database or API)
raw_orders = [
    {"order_id": 1001, "customer_type": "premium", "amount": 250.00, "items": 3, "region": "North"},
    {"order_id": 1002, "customer_type": "standard", "amount": -50.00, "items": 1, "region": "South"},  # Invalid
    {"order_id": 1003, "customer_type": "premium", "amount": 1250.00, "items": 15, "region": "East"},
    {"order_id": 1004, "customer_type": "", "amount": 75.00, "items": 2, "region": "West"},  # Missing type
    {"order_id": 1005, "customer_type": "standard", "amount": 450.00, "items": 6, "region": "North"},
    {"order_id": 1006, "customer_type": "vip", "amount": 3500.00, "items": 25, "region": "North"},
    {"order_id": 1007, "customer_type": "standard", "amount": 0, "items": 0, "region": "South"},  # Invalid
    {"order_id": 1008, "customer_type": "premium", "amount": 890.00, "items": 8, "region": "West"},
]



In [5]:
# =============================================================================
# STEP 1: DATA VALIDATION (Using Conditions in Loops)
# =============================================================================

print("\nSTEP 1: VALIDATING ORDERS")
print("-" * 40)

valid_orders = []
rejected_orders = []

for order in raw_orders:
    # Check multiple validation rules
    is_valid = True
    rejection_reasons = []
    
    # Rule 1: Amount must be positive
    if order["amount"] <= 0:
        is_valid = False
        rejection_reasons.append("invalid amount")
    
    # Rule 2: Must have items
    if order["items"] <= 0:
        is_valid = False
        rejection_reasons.append("no items")
    
    # Rule 3: Must have customer type
    if not order["customer_type"]:
        is_valid = False
        rejection_reasons.append("missing customer type")
    
    # Process based on validation
    if is_valid:
        valid_orders.append(order)
        print(f"✓ Order {order['order_id']}: Valid")
    else:
        order["rejection_reasons"] = ", ".join(rejection_reasons)
        rejected_orders.append(order)
        print(f"✗ Order {order['order_id']}: Rejected ({order['rejection_reasons']})")

print(f"\nValidation Results:")
print(f"  Valid orders: {len(valid_orders)}")
print(f"  Rejected orders: {len(rejected_orders)}")




STEP 1: VALIDATING ORDERS
----------------------------------------
✓ Order 1001: Valid
✗ Order 1002: Rejected (invalid amount)
✓ Order 1003: Valid
✗ Order 1004: Rejected (missing customer type)
✓ Order 1005: Valid
✓ Order 1006: Valid
✗ Order 1007: Rejected (invalid amount, no items)
✓ Order 1008: Valid

Validation Results:
  Valid orders: 5
  Rejected orders: 3


In [6]:
# =============================================================================
# STEP 2: CATEGORIZING AND ENRICHING DATA
# =============================================================================

print("\nSTEP 2: CATEGORIZING ORDERS")
print("-" * 40)

for order in valid_orders:
    # Categorize by order size
    if order["amount"] >= 1000:
        order["size_category"] = "Large"
        order["processing_priority"] = "High"
    elif order["amount"] >= 500:
        order["size_category"] = "Medium"
        order["processing_priority"] = "Normal"
    else:
        order["size_category"] = "Small"
        order["processing_priority"] = "Low"
    
    # Apply customer-specific discounts
    if order["customer_type"] == "vip":
        order["discount_rate"] = 0.20
    elif order["customer_type"] == "premium":
        order["discount_rate"] = 0.10
    else:
        order["discount_rate"] = 0.05
    
    # Calculate shipping based on region and size
    base_shipping = 10.00
    
    if order["region"] in ["North", "South"]:
        region_multiplier = 1.0
    else:  # East, West
        region_multiplier = 1.5
    
    if order["items"] > 10:
        size_multiplier = 1.5
    elif order["items"] > 5:
        size_multiplier = 1.2
    else:
        size_multiplier = 1.0
    
    order["shipping_cost"] = base_shipping * region_multiplier * size_multiplier
    
    # Calculate final amount
    discount = order["amount"] * order["discount_rate"]
    order["final_amount"] = order["amount"] - discount + order["shipping_cost"]
    
    print(f"Order {order['order_id']}: {order['size_category']:6} | "
          f"Priority: {order['processing_priority']:6} | "
          f"Final: ${order['final_amount']:,.2f}")


STEP 2: CATEGORIZING ORDERS
----------------------------------------
Order 1001: Small  | Priority: Low    | Final: $235.00
Order 1003: Large  | Priority: High   | Final: $1,147.50
Order 1005: Small  | Priority: Low    | Final: $439.50
Order 1006: Large  | Priority: High   | Final: $2,815.00
Order 1008: Medium | Priority: Normal | Final: $819.00


In [7]:
# =============================================================================
# STEP 3: AGGREGATING INSIGHTS
# =============================================================================

print("\nSTEP 3: GENERATING INSIGHTS")
print("-" * 40)

# Initialize counters
total_revenue = 0
orders_by_region = {"North": 0, "South": 0, "East": 0, "West": 0}
orders_by_priority = {"High": 0, "Normal": 0, "Low": 0}
customer_type_revenue = {"vip": 0, "premium": 0, "standard": 0}

# Process each order for aggregations
for order in valid_orders:
    # Add to total
    total_revenue += order["final_amount"]
    
    # Count by region
    orders_by_region[order["region"]] += 1
    
    # Count by priority
    orders_by_priority[order["processing_priority"]] += 1
    
    # Sum revenue by customer type
    customer_type = order["customer_type"]
    if customer_type in customer_type_revenue:
        customer_type_revenue[customer_type] += order["final_amount"]

# Find best performing region
best_region = None
max_orders = 0
for region, count in orders_by_region.items():
    if count > max_orders:
        max_orders = count
        best_region = region

print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Order Value: ${total_revenue/len(valid_orders):,.2f}")
print(f"Best Region: {best_region} ({max_orders} orders)")

print("\nOrders by Priority:")
for priority, count in orders_by_priority.items():
    print(f"  {priority}: {count}")

print("\nRevenue by Customer Type:")
for ctype, revenue in customer_type_revenue.items():
    if revenue > 0:  # Only show types with revenue
        percentage = (revenue / total_revenue) * 100
        print(f"  {ctype}: ${revenue:,.2f} ({percentage:.1f}%)")




STEP 3: GENERATING INSIGHTS
----------------------------------------
Total Revenue: $5,456.00
Average Order Value: $1,091.20
Best Region: North (3 orders)

Orders by Priority:
  High: 2
  Normal: 1
  Low: 2

Revenue by Customer Type:
  vip: $2,815.00 (51.6%)
  premium: $2,201.50 (40.4%)
  standard: $439.50 (8.1%)


In [8]:
# =============================================================================
# STEP 4: IDENTIFYING ACTION ITEMS
# =============================================================================

print("\nSTEP 4: ACTION ITEMS")
print("-" * 40)

# Check for alerts
alerts = []

# Alert 1: High-value orders needing attention
high_value_orders = []
for order in valid_orders:
    if order["amount"] > 1000:
        high_value_orders.append(order["order_id"])

if high_value_orders:
    alerts.append(f"High-value orders requiring manual review: {high_value_orders}")

# Alert 2: Regions with low activity
for region, count in orders_by_region.items():
    if count < 2:
        alerts.append(f"Low activity in {region} region (only {count} orders)")

# Alert 3: Customer type distribution
vip_count = sum(1 for order in valid_orders if order["customer_type"] == "vip")
if vip_count == 0:
    alerts.append("No VIP orders today - consider targeted marketing")

# Display alerts
if alerts:
    print("⚠️ Alerts:")
    for i, alert in enumerate(alerts, 1):
        print(f"  {i}. {alert}")
else:
    print("✓ No alerts - all metrics within normal range")




STEP 4: ACTION ITEMS
----------------------------------------
⚠️ Alerts:
  1. High-value orders requiring manual review: [1003, 1006]
  2. Low activity in South region (only 0 orders)
  3. Low activity in East region (only 1 orders)
  4. Low activity in West region (only 1 orders)


In [9]:
# =============================================================================
# FLOW CONTROL DEMONSTRATIONS
# =============================================================================

print("\n" + "=" * 60)
print("FLOW CONTROL: BREAK, CONTINUE, PASS, AND ELSE")
print("=" * 60)

# -----------------------------------------------------------------------------
# CONTINUE: Skip specific iterations
# -----------------------------------------------------------------------------
print("\n1. CONTINUE - Skip Certain Items")
print("-" * 40)

print("Processing orders with special handling:")
for order in valid_orders:
    # Skip small orders in this special processing
    if order["size_category"] == "Small":
        print(f"  Skipping small order {order['order_id']}")
        continue  # Jump to next iteration
    
    # This code only runs for Medium and Large orders
    print(f"  Processing {order['size_category']} order {order['order_id']}")
    # ... special processing here ...

# -----------------------------------------------------------------------------
# BREAK: Exit loop early
# -----------------------------------------------------------------------------
print("\n2. BREAK - Stop When Condition Met")
print("-" * 40)

# Find first VIP order (stop searching once found)
vip_order = None
search_count = 0

for order in valid_orders:
    search_count += 1
    print(f"  Checking order {order['order_id']}...")
    
    if order["customer_type"] == "vip":
        vip_order = order
        print(f"  Found VIP order! Stopping search.")
        break  # Exit the loop completely
    
print(f"Searched {search_count} orders out of {len(valid_orders)} total")
if vip_order:
    print(f"VIP Order found: {vip_order['order_id']}")
else:
    print("No VIP orders found")




FLOW CONTROL: BREAK, CONTINUE, PASS, AND ELSE

1. CONTINUE - Skip Certain Items
----------------------------------------
Processing orders with special handling:
  Skipping small order 1001
  Processing Large order 1003
  Skipping small order 1005
  Processing Large order 1006
  Processing Medium order 1008

2. BREAK - Stop When Condition Met
----------------------------------------
  Checking order 1001...
  Checking order 1003...
  Checking order 1005...
  Checking order 1006...
  Found VIP order! Stopping search.
Searched 4 orders out of 5 total
VIP Order found: 1006


In [ ]:
# -----------------------------------------------------------------------------
# ELSE with loops: Runs if loop completes without break
# -----------------------------------------------------------------------------
print("\n3. ELSE with Loops - Completion Check")
print("-" * 40)

# Search for problematic order
threshold = 5000
print(f"Checking for orders over ${threshold}...")

for order in valid_orders:
    if order["amount"] > threshold:
        print(f"  ⚠️ Order {order['order_id']} exceeds threshold!")
        break
else:
    # This runs ONLY if loop completes without hitting break
    print(f" ✓ All orders within threshold of ${threshold}")




3. ELSE with Loops - Completion Check
----------------------------------------
Checking for orders over $5000...
  ✓ All orders within threshold of $5000


In [11]:
# -----------------------------------------------------------------------------
# PASS: Placeholder for empty blocks
# -----------------------------------------------------------------------------
print("\n4. PASS - Placeholder Statement")
print("-" * 40)

for order in valid_orders[:3]:  # Just first 3 for demo
    if order["processing_priority"] == "High":
        # TODO: Implement rush processing
        pass  # Placeholder - does nothing but keeps syntax valid
    elif order["processing_priority"] == "Normal":
        print(f"  Normal processing for {order['order_id']}")
    else:
        # TODO: Implement batch processing for low priority
        pass


4. PASS - Placeholder Statement
----------------------------------------


In [17]:
# -----------------------------------------------------------------------------
# Nested loops with break and continue
# -----------------------------------------------------------------------------
print("\n5. NESTED LOOPS - Complex Flow Control")
print("-" * 40)

# Check for duplicate amounts across regions
regions_to_check = ["North", "South"]
amounts_seen = []

print("Checking for duplicate order amounts:")
for region in regions_to_check:
    print(f"\nRegion: {region}")
    
    for order in valid_orders:
        if order["region"] != region:
            continue  # Skip orders not in current region
        
        if order["amount"] in amounts_seen:
            print(f"  Found duplicate amount ${order['amount']}")
            break  # Stop checking this region
        
        amounts_seen.append(order["amount"])
        print(f"  Order {order['order_id']}: ${order['amount']} (unique)")



5. NESTED LOOPS - Complex Flow Control
----------------------------------------
Checking for duplicate order amounts:

Region: North
  Order 1001: $250.0 (unique)
  Order 1005: $450.0 (unique)
  Order 1006: $3500.0 (unique)

Region: South


In [18]:

# -----------------------------------------------------------------------------
# While loop with break
# -----------------------------------------------------------------------------
print("\n6. WHILE with BREAK - User Input Simulation")
print("-" * 40)

# Simulate processing orders until budget exhausted
budget = 2000
processed = []
remaining_orders = valid_orders.copy()

print(f"Processing orders with budget ${budget}")
while remaining_orders:  # While list is not empty
    order = remaining_orders[0]  # Get first order
    
    if order["final_amount"] > budget:
        print(f"  Cannot afford order {order['order_id']} (${order['final_amount']:.2f})")
        remaining_orders.pop(0)
        continue  # Try next order
    
    # Process the order
    budget -= order["final_amount"]
    processed.append(order)
    remaining_orders.pop(0)
    print(f"  Processed {order['order_id']}, remaining budget: ${budget:.2f}")
    
    if budget < 50:  # Stop if budget too low
        print("  Budget too low, stopping.")
        break

print(f"\nProcessed {len(processed)} orders")
print(f"Remaining budget: ${budget:.2f}")



6. WHILE with BREAK - User Input Simulation
----------------------------------------
Processing orders with budget $2000
  Processed 1001, remaining budget: $1765.00
  Processed 1003, remaining budget: $617.50
  Processed 1005, remaining budget: $178.00
  Cannot afford order 1006 ($2815.00)
  Cannot afford order 1008 ($819.00)

Processed 3 orders
Remaining budget: $178.00


In [19]:
# =============================================================================
# RETURN VS BREAK VS CONTINUE (Function Context)
# =============================================================================

print("\n" + "=" * 60)
print("FLOW CONTROL IN FUNCTIONS")
print("=" * 60)

def process_order_batch(orders, max_errors=2):
    """Demonstrate return vs break vs continue in functions"""
    processed_count = 0
    error_count = 0
    
    print(f"\nProcessing batch of {len(orders)} orders:")
    
    for order in orders:
        # Skip invalid orders
        if order["amount"] <= 0:
            print(f"  Skipping invalid order {order['order_id']}")
            continue  # Skip to next order
        
        # Simulate processing error
        if order["order_id"] % 2 == 0:  # Even order IDs "fail"
            error_count += 1
            print(f"  Error processing {order['order_id']} (error #{error_count})")
            
            if error_count >= max_errors:
                print(f"  Too many errors! Stopping batch.")
                return processed_count  # Exit function entirely
            
            continue  # Skip this order, try next
        
        # Success
        processed_count += 1
        print(f"  ✓ Processed order {order['order_id']}")
    
    return processed_count  # Normal function exit

# Test the function
result = process_order_batch(valid_orders, max_errors=2)
print(f"\nFunction returned: {result} orders processed")




FLOW CONTROL IN FUNCTIONS

Processing batch of 5 orders:
  ✓ Processed order 1001
  ✓ Processed order 1003
  ✓ Processed order 1005
  Error processing 1006 (error #1)
  Error processing 1008 (error #2)
  Too many errors! Stopping batch.

Function returned: 3 orders processed


##### SUMMARY: WHEN TO USE EACH FLOW CONTROL


1. **CONTINUE** - Skip rest of current iteration, go to next ("Skip this one, but keep going")

   Use when: You want to skip certain items but keep processing others

   Example: Skip invalid records in data cleaning

2. **BREAK** - Exit the loop completely ("I found what I need, stop everything")

   Use when: You've found what you're looking for or hit a stop condition

   Example: Stop searching once target is found

3. **PASS** - Do nothing, placeholder  ("I'll come back to this later")

   Use when: Syntax requires a statement but logic isn't implemented yet

   Example: Empty except blocks or TODO sections

4. **ELSE** (with loops) - Run code if loop completed normally ("If we made it through everything...")

   Use when: You need to know if loop finished without break

   Example: Verify all items were processed successfully

5. **RETURN** - Exit function and optionally return value

   Use when: Function's job is complete or error requires exit

   Example: Early exit from validation function

**LOOP CHOICE GUIDE:**

- **FOR loop:** When you know what to iterate over (lists, ranges, etc.)

- **WHILE loop**: When you have a condition but don't know iterations

- **Nested loops**: When processing multi-dimensional data

**PERFORMANCE TIPS**:

- Use 'break' to exit early and save processing time

- Use 'continue' to skip unnecessary processing

- Consider list comprehensions for simple filtering



**PRACTICAL EXERCISES FOR FLOW CONTROL:**

- Find and Stop: Search for first order over $1000 and stop

- Skip and Process: Process all orders except cancelled ones

- Count Until Limit: Process orders until budget exhausted

- Validate All: Check if all records pass validation (use else)





In [20]:


# =============================================================================
# BONUS: USING PANDAS FOR COMPARISON
# =============================================================================

print("\n" + "=" * 60)
print("BONUS: Same Analysis with Pandas (Preview)")
print("=" * 60)

# Convert to DataFrame
df = pd.DataFrame(valid_orders)

# Quick aggregations
print("\nPandas Summary:")
print(f"Total orders: {len(df)}")
print(f"Total revenue: ${df['final_amount'].sum():,.2f}")
print(f"Average order: ${df['final_amount'].mean():,.2f}")

print("\nOrders by region:")
print(df['region'].value_counts())

print("\nRevenue by customer type:")
revenue_by_type = df.groupby('customer_type')['final_amount'].sum().round(2)
print(revenue_by_type)

print("\n💡 Notice how pandas makes aggregations easier!")
print("   We'll cover this in detail in upcoming weeks.")
print("   For now, focus on understanding loop logic!")


BONUS: Same Analysis with Pandas (Preview)

Pandas Summary:
Total orders: 5
Total revenue: $5,456.00
Average order: $1,091.20

Orders by region:
region
North    3
East     1
West     1
Name: count, dtype: int64

Revenue by customer type:
customer_type
premium     2201.5
standard     439.5
vip         2815.0
Name: final_amount, dtype: float64

💡 Notice how pandas makes aggregations easier!
   We'll cover this in detail in upcoming weeks.
   For now, focus on understanding loop logic!
